# Assignment 1

## Part 1: Using web-scraping to gather data

We wish to find the list of researchers that have joined the most important scientific conference in Computational Social Science in 2019.

We find them by webscraping the programmes of the 2019 edition via the flowwing links:  
>Oral presentations: https://2019.ic2s2.org/oral-presentations/    
> Poster presentations: https://2019.ic2s2.org/posters/    

Summary:
>We found 888 unique researchers in 2019
>Explain one or two decisions you took during the web-scraping exercise, for 2019 or any other year. Why did you take this choice? How might your decision impact the final number of authors?
- We took the decision to include authors that was listed as "no presentation" since we figured that we were looking for who were social data scientists and thus a cancelled presenter would still be valid.
- Futheremore, we decided to do an error search for names longer than 25 characters which helped us find about 10-15 more authors who would have ha

In [529]:
from bs4 import BeautifulSoup ##A package to work with HTML data
import requests #A package to make HTTP requests
import re

In [530]:
#Part 1: webscraping
#Oral Presenters, setup link for html webscraping
LINK = "https://2019.ic2s2.org/oral-presentations/"
r = requests.get(LINK)
soup_oral = BeautifulSoup(r.content)

In [531]:
#Extract all oral presenters form url
data = soup_oral.findAll("p") #extract the part with the relevant data
presenters_all_topics = []
for topics in data[3:]: #loop over all the topic sections
    presenters = []
    for presentation in topics: #loop over each presentation in each topic
        try:
            presenters_NoClock = presentation.split(" – ")[2] #get the string of names after each time slot
        except:
            pass
        presenters.append(presenters_NoClock)
    prenters_topic = (", ".join([lecture.split(".")[0] for lecture in presenters])).replace("<","").replace("–","") #only include the names of the presenters and not the topic
    prenters_topic = prenters_topic.split(", ") #turn into single list with all the names in the rest of the lines
    presenters_all_topics.append(prenters_topic)
    oral_presenters_2019 = [item for sublist in presenters_all_topics for item in sublist]

In [532]:
#Poster Presenters
#Setup link for html webscraping
LINK = "https://2019.ic2s2.org/posters/"
r_poster = requests.get(LINK)
soup_post = BeautifulSoup(r_poster.content)

In [533]:
table = soup_post.find("div",{"class":"col-md-8 page-content-wrap"}) #Get the section that contains all the data. We call it a table

In [534]:
poster_researchers_all = []
for day in table.findAll("ul"): #loop over both days
    for poster in day.findAll("li"): #loop over each poster  presentation
        if str(poster)[:12] != "<li><strong>": #from each poster presentation extract only the names and not the topic
            poster_researh_grouped = str(poster).split("<span>")[0][4:].replace("</li>","") #reformat the names to fit in a single list and exclude commas and "and"
            poster_researchers = [item for sublist in [team.split(" and ") for team in poster_researh_grouped.split(", ")] for item in sublist]
            poster_researchers_all.extend(poster_researchers)

In [535]:
all_presenters = poster_researchers_all + oral_presenters_2019

In [536]:
for i,presenter in enumerate(all_presenters): #Some semi-manual error detection for names over a given lenght. Unhash to perform a few error corrections due to inconsistency in typing of the website
    if len(presenter) >25:
        print(i,presenter)
#all_presenters.remove('Keyword-Based Procedure for Identifying Research-Relevant Text</strong>')
#all_presenters.remove('Abhishek Samantray,Massimo Riccaboni')
#all_presenters.extend(['Abhishek Samantray','Massimo Riccaboni'])
#all_presenters.remove('(Moved to 3D Text Analysis) Ivan Smirnov')
#all_presenters.remove('No presentation: Vincent Antonio Traag')
#all_presenters.extend(['Ivan Smirnov','Vincent Antonio Traag'])
#all_presenters.remove('Christos Nicolaides,Luis Cueto-Felgueroso')
#all_presenters.extend(['Christos Nicolaides','Luis Cueto-Felgueroso'])
#all_presenters.remove('No presentation: Eelke Heemskerk')
#all_presenters.extend(['Eelke Heemskerk'])
#all_presenters.extend(['Rens Wilderom'])
#all_presenters.remove('No presentation (cancelled)')
#all_presenters.remove('No presentation: Youyou Wu')
#all_presenters.extend('Youyou Wu')
#all_presenters = sorted(all_presenters)[:6]
#del all_presenters[:6]


4 Gianmarco De Francisci Morales
50 Zachary Steinert-Threlkeld
62 Gianmarco De Francisci Morales
174 Mariken A.C.G. van der Velden
191 Diego Fregolente Mendes de Oliveira
206 Christos Nicolaides,Luis Cueto-Felgueroso
227 and Rens Wilderom<br/>
<strong>Separating the Wheat from the Chaff: A Topic-
228 Keyword-Based Procedure for Identifying Research-Relevant Text</strong>
255 Gianmarco De Francisci Morales
805 No presentation (cancelled)
806 No presentation (cancelled)
1006 Gianmarco De Francisci Morales
1010 Gianmarco De Francisci Morales
1043 No presentation: Youyou Wu
1046 No presentation: Youyou Wu
1068 Hartmut Wessler,Scott Althaus
1074 Hartmut Wessler,Scott Althaus
1231 Abhishek Samantray,Massimo Riccaboni
1232 Abhishek Samantray,Massimo Riccaboni
1369 No presentation (cancelled)
1370 No presentation (cancelled)
1557 (Moved to 3D Text Analysis) Ivan Smirnov
1558 (Moved to 3D Text Analysis) Ivan Smirnov
1819 No presentation: Vincent Antonio Traag
1820 No presentation: Vincent Anton

In [492]:
#Export unique names to .txt file
import codecs
all_presenters = sorted(set(all_presenters)) #Convert to a set data type to only get unique names and sort so the 4 time slot error can be manually removed
with codecs.open('all_presenters_2019.txt', 'w', "utf-8") as f:
    f.write(str(all_presenters))

print(len(sorted(set(all_presenters))),"unique researcher found as presenters for the 2019 conference and uploaded to a txt file")

888 unique researcher found as presenters for the 2019 conference and uploaded to a txt file


## Part 2: Getting data from the Semantic Scholar API

> * Consider the list of author ids you have found in Week 2, Part 3, first excercise. For each author, use the Academic Graph API to find:
>    - their _aliases_
>    - their _name_
>    - their _papers_, where for each paper we want to retain: 
>        -  _title_ 
>        -  _abstract_ 
>        -  the _year_ of publication
>        -  the _externalIds_ (this is because there are universal identifiers for scientific works called DOI that we can use across platforms)
>        -  _s2FieldsOfStudy_ the fields of study
>        - _citationCount_ the number of times that this paper was cited    
> * Create three dataframe to store the data you have collected. 
>    
>    - **Author dataset:** in the author dataset, one raw is one unique author, and each row contains the following information: 
>        - *authorId*: (str) the id of the author
>        - *name*: (str) the name of the author
>        - *aliases*: (list) the aliases of the author
>        - *citationCount*: (int) the total number of citations received by an author
>        - *field*: (str) the _s2FieldsOfStudy_ that occurs most times across an author's papers (you should first obtain the *category* for each _s2FieldsOfStudy_)
>    - **Paper dataset:** in the paper dataset, one row is one unique paper, and each row contains the following information:
>        - *paperId*: (str) the id of the paper
>        - *title*: (str) the title of the paper
>        - *year*: (int) the year of publication
>        - *externalId.DOI:* (str) the DOI of the paper
>        - *citationCount*: (int) the number of citations
>        - *fields*: (list) the fields included in the paper (you should first obtain the *category* for each _s2FieldsOfStudy_)
>        - *authorIds:* (list) this is a list of *author Ids*, including all the authors of this paper that are in our author dataset
>    - **Paper abstract dataset:** in the paper abstract dataset, one row is one unique paper, and each row contains the following information: 
>        - *paperId*: (str) the id of the paper
>        - *abstract*: (str) the abstract of the paper    
>  (Note: we keep the abstract separate to keep the size of files more manageable)


1. Share the number of authors you will use as starting point in this exercises. Add a comment clarifying how many IC2S2 editions you included and if the collaborators were included or not.
2. Share the code you have used to solve the exercise above.
3. How long is your final _Author_ dataframe? How long is your final _Paper_ dataframe? 

(**Note**: If you did not manage to get all the years or all the authors' collaborators, you can still follow the exercise. Just remember to clarify your starting point.)



## Part 3: Law of large numbers.

As we have discussed in the lecture, one impact of heavy tails is that sample averages can be poor estimators of the underlying mean of the distribution.
To understand this point better, recall [the Law of Large Numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers).  Consider a sample of IID variables $ X_1, \ldots, X_n $ from the same distribution $ F $ with finite expected value $ \mathbb E |X_i| =  \int x F(dx)  = \mu $.

According to the law, the mean of the sample $ \bar X_n := \frac{1}{n} \sum_{i=1}^n X_i $ satisfies
<a id='equation-lln-as2'></a>
$$
\bar X_n \to \mu \text{ as } n \to \infty 
$$

This basically tell us that if we have a large enough sample, the sample mean will converge to the population mean. 

The condition that $ \mathbb E | X_i | $ is finite holds in most cases but can fail if the distribution $ F $ is very heavy tailed. Further, even when $ \mathbb E | X_i | $ is finite, the variance of a heavy tailed distribution can be so large that the sample mean will converge very slowly to the population mean. We will look into this in the following exercise. 


> 1. Sample __N=10,000__ data points from a [Gaussian Distribution](https://en.wikipedia.org/wiki/Normal_distribution) with parameters $\mu = 0 $ and $\sigma = 4$, using the [`np.random.standard_normal()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.standard_normal.html) function. Store your data in a numpy array $\mathbf{X}$. 
> 2. Create a figure. 
>    - Plot the distribution of the data in $\mathbf{X}$. 
> 3. Compute the cumulative average of $\mathbf{X}$ (you achieve this by computing $average(\{\mathbf{X}[0],..., \mathbf{X}[i-1]\})$ for each index $i \in [1, ..., N+1]$  ). Store the result in an array. 
> 4. In a similar way, compute the cumulative standard error of $\mathbf{X}$. __Note__: the standard error of a sample is defined as $ \sigma_{M} = \frac{\sigma}{\sqrt(n)} $, where $\sigma$ is the sample standard deviation and $n$ is the sample size. Store the result in an array. 
> 5. Compute the values of the distribution mean and median using the formulas you can find on the [Wikipedia page of the Gaussian Distribution](https://en.wikipedia.org/wiki/Normal_distribution)  
> 6. Create a figure. 
>     - Plot the cumulative average computed in point 3. as a line plot (where the x-axis represent the size of the sample considered, and the y-axis is the average).
>     - Add errorbars to each point in the graph with width equal to the standard error of the mean (the one you computed in point 4). 
>     - Add a horizontal line corresponding to the distribution mean (the one you found in point 5).
> 7.  Compute the cumulative median of $\mathbf{X}$ (you achieve this by computing $median(\{\mathbf{X}[0],..., \mathbf{X}[i-1]\})$ for each index $i \in [1, ..., N+1]$). Store the result in an array. 
> 8. Create a figure. 
>    - Plot the cumulative median computed in point 7. as a line plot (where the x-axis represent the size of the sample considered, and the y-axis is the average).
>    - Add a horizontal line corresponding to the distribution median (the one you found in point 5).
>    - _Optional:_ Add errorbars to your median line graph, with width equal to the standard error of the median. You can compute the standard error of the median [via bootstrapping](https://online.stat.psu.edu/stat500/book/export/html/619). 
> 9. Now sample __N = 10,000__ data points from a [Pareto Distribution](https://en.wikipedia.org/wiki/Pareto_distribution) with parameters $x_m=1$ and $\alpha=0.5$ using the [`np.random.pareto()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.pareto.html) function, and store it in a numpy array. (_Optional:_ Write yourself the function to sample from a Pareto distribution using the [_Inverse Transform Sampling method_](https://en.wikipedia.org/wiki/Inverse_transform_sampling))
> 10. Repeat points 2 to 8 for the Pareto Distribution sample computed in point 9. 
> 11. Now sample __N = 10,000__ data points from a [Lognormal Distribution](https://en.wikipedia.org/wiki/Log-normal_distribution) with parameters $\mu=0$ and $\sigma=4$ using the [`np.random.standard_normal()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.standard_normal.html) function, and store it in a numpy array. 
> 12. Repeat points 2 to 8 for the Lognormal Distribution sample computed in point 11. 
> 13. Now, consider the array collecting the citations of papers from 2009 you created in Exercise 3, point 1. First, compute the mean and median number of citations for this population. Then, extract a random sample of __N=10,000__ papers.  
> 14. Repeat points 2,3,4,6,7 and 8 above for the paper citation sample prepared in point 13. 

> Answer the following questions:
(__Hint__: I suggest you plot the graphs above multiple times for different random samples, to get a better understanding of what is going on)

>    -  Compare the evolution of the cumulative average for the Gaussian, Pareto and LogNormal distribution. What do you observe? Would you expect these results? Why?
>    - Compare the cumulative median vs the cumulative average for the three distributions. What do you observe? Can you draw any conclusions regarding which statistics (the mean or the median) is more usfeul in the different cases? 
>    - Consider the plots you made using the citation count data in point 14. What do you observe? What are the implications? 
>    - What do you think are the main take-home message of this exercise? 